In [1]:
import pandas as pd
import numpy as np

In [2]:
events = pd.read_csv('blitz/course-217-events.csv')
structure = pd.read_csv('blitz/course-217-structure.csv')

In [3]:
struct = np.array(structure[['step_id', 'module_position', 'lesson_position', 'step_position']])

In [4]:
struct_order = struct[:,3] + struct[:,2] * 100 + struct[:,1] * 1000

In [5]:
steps = struct[struct_order.argsort()][:,0]
score = np.zeros_like(steps, dtype=np.float64)

In [6]:
events = np.array(events) # user_id,action,step_id,time
structure = np.array(structure) # course_id,module_id,module_position,lesson_id,
                                # lesson_position,step_id,step_position,step_type,step_cost

In [7]:
users = np.unique(events[:,0])

In [8]:
max_time = np.max(events[:,3])
min_time = np.min(events[:,3])

user_step = {} # min,max,disc,ret

for step in steps:
    for user in users:
        user_step[(user, step)] = [max_time, min_time, False, False]

In [9]:
for event in events:
    user_id, _, step_id, time = event
    
    user_step[(user_id, step_id)][2] = True    
    user_step[(user_id, step_id)][0] = min(user_step[(user_id, step_id)][0], time)
    user_step[(user_id, step_id)][1] = max(user_step[(user_id, step_id)][1], time)

In [10]:
for event in events:
    user_id, _, step_id, time = event
    
    ind = np.where(steps[1:] == step_id)[0]
    
    if ind.size == 0:
        continue
    
    prev_step_id = steps[ind[0]]
    
    if user_step[(user_id, prev_step_id)][0] < time < user_step[(user_id, prev_step_id)][1]:
        user_step[(user_id, prev_step_id)][3] = True

In [11]:
for i, step in enumerate(steps):
    
    all_users = 0
    ret_users = 0
    
    for user in users:
        _, _, disc, ret = user_step[(user, step)]
        if disc:
            all_users += 1
        if ret:
            ret_users += 1
        
    if all_users == 0:
        score[i] = 0
    else:
        score[i] = 100 * ret_users / all_users

In [12]:
indices = score.argsort()
steps = steps[indices]
steps[-10:]

array([39717, 39740, 39735, 41686, 38872, 42593, 41481, 41097, 41604, 41684], dtype=int64)